In [61]:
import uuid
from langserve import RemoteRunnable

import os
import sys

%env REDIS_OM_URL=redis://10.0.1.6:6379/0
%env REDIS_HOST=10.0.1.6
%env REDIS_PORT=6379
%env LANGSERVE_URL=http://10.0.1.6:8000/


platform_id="svaeva-redux"
group_id="consonancia"
conversation_id="sim-consonancia"
interaction_count=0

env: REDIS_OM_URL=redis://10.0.1.6:6379/0
env: REDIS_HOST=10.0.1.6
env: REDIS_PORT=6379
env: LANGSERVE_URL=http://10.0.1.6:8000/


In [62]:
from svaeva_redux.schemas.redis import UserModel

users = UserModel.find(
    (UserModel.group_id == group_id)
    & (UserModel.platform_id == platform_id)
    & (UserModel.interaction_count >= interaction_count)
).all()

ConnectionError: Error while reading from 10.0.1.6:6379 : (54, 'Connection reset by peer')

In [63]:
import asyncio
import logging
import os
import re
from typing import Dict, List
import re
import dotenv
from openai import OpenAI
from openai.types.images_response import ImagesResponse
from rich.logging import RichHandler


import base64
import io
import json
import logging
import os
from datetime import datetime

import dotenv
import httpx
import redis
from fastapi import HTTPException


def update_user_with_image(user_id):
    user = UserModel.get(user_id)
    model = "gpt-4"
    temperature = 0.7
    system_prompt = SYSTEM_DALLE_PROMPT
    user_content = text
    client = OpenAI()
    try:
        # Chat response for DALL-E prompt
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content},
            ],
        )
        prompt = response.choices[0].message.content
        prompt = re.sub(r'"', "", prompt)

        # Generate image from DALL-E prompt
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )

    except Exception as e:
        logging.error(f"Error: {e}")


    image_url = response.data[0].url

    try:
        # response = requests.get(url)
        # if response.status_code != 200:
        #     raise HTTPException(status_code=400, detail="Failed to download image")
        client = httpx.Client() 
        response = client.get(image_url)
        if response.status_code != 200:
            raise HTTPException(status_code=400, detail="Failed to download image")
        # Convert the image to bytes
        image_bytes = io.BytesIO(response.content)
        # Encode the image bytes as a base64 string
        image_base64 = base64.b64encode(image_bytes.getvalue()).decode("utf-8")
    except Exception as e:
        image_base64 = ""
        logging.error(f"Error: {e}")

    user.avatar_image_prompt = prompt
    user.avatar_image_bytes = image_base64
    
    
    return user

In [66]:
from svaeva_evaluation.simulation.consonancia import retrieve_redis_windowed_chat_history_as_text
from svaeva_evaluation.prompts.consonancia import arthritis, breakup, chronic_hip_pain, disc_hernia, pots
prompts = dict()
# prompts["disc_hernia"] = disc_hernia
# prompts["arthritis"] = arthritis
prompts["chronic_hip_pain"] = chronic_hip_pain
prompts["pots"] = pots
prompts["breakup"] = breakup

key_prefix = f"{platform_id}_{conversation_id}:"
for key, value in prompts.items():
    condition = key
    for i in range(1, 7):
        print(condition, i)
        user_id = f"sim-consonancia-{condition}-{str(i).zfill(3)}"
        print(user_id)
        url = str(os.getenv("REDIS_OM_URL"))
        chat_history_length = 40
        text = retrieve_redis_windowed_chat_history_as_text(
            user_id, url, key_prefix, chat_history_length=chat_history_length
        )

        user = update_user_with_image(user_id)

        user.save()

        # compute_conversation_embedding_and_update_user(
        #     user_id,
        #     os.environ["REDIS_OM_URL"],
        #     key_prefix,
        #     chat_history_length=30,
        #     model="text-embedding-3-large",
        # )

chronic_hip_pain 1
sim-consonancia-chronic_hip_pain-001


[18:59:31] INFO     HTTP Request: POST                                        ]8;id=988837;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=452516;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    https://api.openai.com/v1/chat/completions "HTTP/1.1 200                 
                    OK"

[18:59:45] INFO     HTTP Request: POST                                        ]8;id=230704;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=975992;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    https://api.openai.com/v1/images/generations "HTTP/1.1                   
                    200 OK"

[18:59:46] INFO     HTTP Request: GET https://oaidalleapiprodscus.blob.core.w ]8;id=631109;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=762600;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    indows.net/private/org-77FRazywqMikL1SL1V5OQ0dm/user-gR3q                
                    3d0UmWWt3X6p4JlCQBwq/img-y7XD1kVF4fyYQ8rHKAK1nFvh.png?st=                
                    2024-05-01T16%3A59%3A45Z&se=2024-05-01T18%3A59%3A45Z&sp=r                
                    &sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaa                
                    dede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e                
                    -a814-9c849652bcb3&skt=2024-04-30T20%3A08%3A42Z&ske=2024-                
                    05-01T20%3A08%3A42Z&sks=b&skv=2021-08-06&sig=nsABi6b2ZENZ                
                    CVUBBYvgbdFOwZYZZq/TPwH2gbCPP3M%3D "HTTP/1.1 200 OK"

chronic_hip_pain 2
sim-consonancia-chronic_hip_pain-002


...

[19:07:48] INFO     HTTP Request: GET https://oaidalleapiprodscus.blob.core.w ]8;id=879508;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=216957;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    indows.net/private/org-77FRazywqMikL1SL1V5OQ0dm/user-gR3q                
                    3d0UmWWt3X6p4JlCQBwq/img-FfMUGqHMDPGrjOcc0MTt9l9f.png?st=                
                    2024-05-01T17%3A07%3A47Z&se=2024-05-01T19%3A07%3A47Z&sp=r                
                    &sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaa                
                    dede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e                
                    -a814-9c849652bcb3&skt=2024-04-30T19%3A47%3A34Z&ske=2024-                
                    05-01T19%3A47%3A34Z&sks=b&skv=2021-08-06&sig=eOGNU8MLU5uq                
                    zy59ep8MzsXOVaPs9tpgMXyWMtbdTs8%3D "HTTP/1.1 200 OK"

breakup 6
sim-consonancia-breakup-006


[19:08:03] INFO     HTTP Request: POST                                        ]8;id=284245;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=658028;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    https://api.openai.com/v1/chat/completions "HTTP/1.1 200                 
                    OK"

[19:08:21] INFO     HTTP Request: POST                                        ]8;id=509269;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=967020;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    https://api.openai.com/v1/images/generations "HTTP/1.1                   
                    200 OK"

[19:08:22] INFO     HTTP Request: GET https://oaidalleapiprodscus.blob.core.w ]8;id=956715;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=53552;file:///Users/eric/Library/Caches/pypoetry/virtualenvs/svaeva-evaluation-GkJecWxE-py3.11/lib/python3.11/site-packages/httpx/_client.py#1026\1026]8;;\
                    indows.net/private/org-77FRazywqMikL1SL1V5OQ0dm/user-gR3q                
                    3d0UmWWt3X6p4JlCQBwq/img-LxbQIbZtbtkUg0aT64CAewwd.png?st=                
                    2024-05-01T17%3A08%3A21Z&se=2024-05-01T19%3A08%3A21Z&sp=r                
                    &sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaa                
                    dede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e                
                    -a814-9c849652bcb3&skt=2024-05-01T12%3A32%3A15Z&ske=2024-                
                    05-02T12%3A32%3A15Z&sks=b&skv=2021-08-06&sig=q8ITynCIR304                
                    WfJacjsrzWVCQo1QNKe5d%2BG2DB5VEGE%3D "HTTP/1.1 200 OK"